In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

# bag of wordsを作成するためのライブラリ
from gensim import corpora, matutils
# TF-IDFを作成するためのライブラリ
from gensim import models

# progress
from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook

C:\Anaconda\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df = pd.read_csv('all_text_wakatigaki.csv',encoding='utf-8')
df.head()

,surface,yomi,original,type,katsuyoukei,katsuyougata,内容
0,neologd,ネオログディー,NEologd,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
1,_,_,_,記号-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
2,tagger,タガー,tagger,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
3,.,.,.,記号-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
4,parse,パース,Parse,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...


In [3]:
# df.groupbyでグループ化された[内容]毎の[original]に対して文字列のリスト（配列）を一つの文字列に連結する「join」関数を各行（各要素）に対して実行する
# join関数の使い方: '間に挿入する文字列'.join([連結したい文字列のリスト])
text_wakati_df = df.groupby('内容')['original'].apply(lambda x: ' '.join(map(str, x))).reset_index()

# カラム名を適切な名前に変更
text_wakati_df = text_wakati_df.rename(columns={'original':'wakati_words'})

# 確認
text_wakati_df

,内容,wakati_words
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd _ tagger . Parse ( text ) で 各 単語 の 原形 ...
1,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、...,まずは Parse () で 分かち書き する た 単語 群 は 1つ の 文字列 型 に ...
2,原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される,原形 、 品詞 など の 間 に は 「 \ t 」 が 、 分かち書き する れる た 単...
3,次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid...,次に 邪魔 だ 文字 「\ t 」 を 省く たい 。 よって 、 split ('\ t ...


In [4]:
wakati_words_list = text_wakati_df['wakati_words'].str.split().tolist()
wakati_words_list

[['NEologd',
  '_',
  'tagger',
  '.',
  'Parse',
  '(',
  'text',
  ')',
  'で',
  '各',
  '単語',
  'の',
  '原形',
  '、',
  '品詞',
  'など',
  'が',
  '1',
  '行',
  'で',
  '連続',
  'する',
  'て',
  '表示',
  'する',
  'れる'],
 ['まずは',
  'Parse',
  '()',
  'で',
  '分かち書き',
  'する',
  'た',
  '単語',
  '群',
  'は',
  '1つ',
  'の',
  '文字列',
  '型',
  'に',
  'なる',
  'て',
  'いる',
  'ので',
  '「',
  '\\',
  'n',
  '」',
  'で',
  '区切る',
  '、',
  'リスト',
  '型',
  'に',
  'する'],
 ['原形',
  '、',
  '品詞',
  'など',
  'の',
  '間',
  'に',
  'は',
  '「',
  '\\',
  't',
  '」',
  'が',
  '、',
  '分かち書き',
  'する',
  'れる',
  'た',
  '単語',
  'と',
  '単語',
  'の',
  '区切り',
  'に',
  'は',
  '「',
  '\\',
  'n',
  '」',
  'が',
  '表示',
  'する',
  'れる'],
 ['次に',
  '邪魔',
  'だ',
  '文字',
  '「\\',
  't',
  '」',
  'を',
  '省く',
  'たい',
  '。',
  'よって',
  '、',
  'split',
  "('\\",
  't',
  "')",
  'を',
  '使用',
  'する',
  'たい',
  'が',
  'wakatid',
  '_',
  'text',
  'は',
  '文字列',
  '型',
  'で',
  'は',
  'ない',
  'リスト',
  '型',
  'の',
  'ため',
  'split',
  '関数',
  'を'

In [5]:
# bag of words作成準備
# 全種類の単語のIDを付与した辞書の作成
corpus_dic = corpora.Dictionary(wakati_words_list)

In [6]:
# 各文章の単語リストをコーパス（辞書の単語IDと単語の出現回数）リストに変換
# 辞書オブジェクトからdoc2bow関数を呼び出し、引数に単語リストを指定することで辞書に基づいたコーパスを取得できる
# コーパスとは、文章に含まれる単語を、辞書の単語IDと単語の出現回数の組み合わせに変換したもの
# bag of wordsに変換するときは変換元のデータ番号が行列の行番号に、単語IDの番号が行列の列番号に、出現回数が行列の値に該当
corpus_list = [corpus_dic.doc2bow(wakati) for wakati in wakati_words_list]

In [7]:
# TF-IDFモデルの作成
# models.TfidfModel関数はコーパスリストをTF-IDF値に変換するオブジェクトを生成する関数
# 引数にコーパスリストを指定し、normalize引数をTrueにするとTF-IDF値をL2ノルムによって正規化した値に変換するオブジェクトになる
tfidf_model = models.TfidfModel(corpus_list, normalize=True)

In [8]:
# corpusにTF-IDFを適用
corpus_list_tfidf = tfidf_model[corpus_list]
wakati_tfidf_matrix = matutils.corpus2csc(corpus_list_tfidf)

In [9]:
print(wakati_tfidf_matrix.toarray())

[[ 0.29477648  0.          0.          0.        ]
 [ 0.29477648  0.          0.          0.        ]
 [ 0.29477648  0.          0.          0.        ]
 [ 0.29477648  0.          0.          0.        ]
 [ 0.29477648  0.          0.          0.        ]
 [ 0.14738824  0.14113122  0.          0.        ]
 [ 0.14738824  0.          0.          0.07687628]
 [ 0.29477648  0.          0.          0.        ]
 [ 0.14738824  0.          0.          0.07687628]
 [ 0.          0.          0.          0.        ]
 [ 0.06117165  0.          0.13333597  0.03190654]
 [ 0.          0.          0.          0.        ]
 [ 0.14738824  0.14113122  0.          0.        ]
 [ 0.12234329  0.11714949  0.          0.03190654]
 [ 0.14738824  0.          0.16063123  0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.14738824  0.          0.32126245  0.        ]
 [ 0.06117165  0.05857475  0.13333597  0.        ]
 [ 0.14738824  0.          0.16063123  0.        ]
 [ 0.29477648  0.          0.  

In [10]:
print(wakati_tfidf_matrix.toarray().T)

[[ 0.29477648  0.29477648  0.29477648  0.29477648  0.29477648  0.14738824
   0.14738824  0.29477648  0.14738824  0.          0.06117165  0.
   0.14738824  0.12234329  0.14738824  0.          0.14738824  0.06117165
   0.14738824  0.29477648  0.14738824  0.29477648  0.14738824  0.29477648
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.14113122
   0.          0.          0.          0.          0.          0.
   0.14113122  0.11714949  0.          0.          0.          0.05857475
   0.          0.    

In [11]:
for i in corpus_list_tfidf:
    for j in i:
        print(corpus_dic[j[0]], j[1])

( 0.294776477035
) 0.294776477035
. 0.294776477035
1 0.294776477035
NEologd 0.294776477035
Parse 0.147388238518
_ 0.147388238518
tagger 0.294776477035
text 0.147388238518
が 0.0611716459375
て 0.147388238518
で 0.122343291875
など 0.147388238518
れる 0.147388238518
単語 0.0611716459375
原形 0.147388238518
各 0.294776477035
品詞 0.147388238518
行 0.294776477035
表示 0.147388238518
連続 0.294776477035
Parse 0.141131216478
て 0.141131216478
で 0.117149494315
単語 0.0585747471574
() 0.282262432957
1つ 0.282262432957
\ 0.141131216478
n 0.141131216478
「 0.141131216478
」 0.0585747471574
いる 0.282262432957
た 0.141131216478
なる 0.282262432957
に 0.282262432957
ので 0.282262432957
は 0.0585747471574
まずは 0.282262432957
リスト 0.141131216478
分かち書き 0.141131216478
区切る 0.282262432957
型 0.282262432957
文字列 0.141131216478
群 0.282262432957
が 0.133335965657
など 0.160631227164
れる 0.321262454328
単語 0.133335965657
原形 0.160631227164
品詞 0.160631227164
表示 0.160631227164
\ 0.321262454328
n 0.160631227164
「 0.321262454328
」 0.133335965657
た 0.160

In [12]:
corpus_list

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 2),
  (12, 1),
  (13, 2),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1)],
 [(5, 1),
  (9, 1),
  (11, 2),
  (12, 1),
  (13, 2),
  (15, 1),
  (17, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 2),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 2),
  (41, 1),
  (42, 1)],
 [(9, 2),
  (10, 2),
  (11, 2),
  (14, 1),
  (15, 2),
  (16, 2),
  (17, 2),
  (18, 1),
  (20, 1),
  (22, 1),
  (26, 2),
  (27, 1),
  (28, 2),
  (29, 2),
  (31, 1),
  (33, 2),
  (35, 2),
  (38, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1)],
 [(6, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (13, 1),
  (15, 1),
  (29, 1),
  (35, 2),
  (37, 1),
  (40, 2),
  (41, 1),
  (43, 2),
  (47, 1),
  (48, 1),
  (49, 2),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 2),
 

In [14]:
print(corpus_dic.token2id)

{'(': 0, ')': 1, '.': 2, '1': 3, 'NEologd': 4, 'Parse': 5, '_': 6, 'tagger': 7, 'text': 8, '、': 9, 'が': 10, 'する': 11, 'て': 12, 'で': 13, 'など': 14, 'の': 15, 'れる': 16, '単語': 17, '原形': 18, '各': 19, '品詞': 20, '行': 21, '表示': 22, '連続': 23, '()': 24, '1つ': 25, '\\': 26, 'n': 27, '「': 28, '」': 29, 'いる': 30, 'た': 31, 'なる': 32, 'に': 33, 'ので': 34, 'は': 35, 'まずは': 36, 'リスト': 37, '分かち書き': 38, '区切る': 39, '型': 40, '文字列': 41, '群': 42, 't': 43, 'と': 44, '区切り': 45, '間': 46, "')": 47, "('\\": 48, 'split': 49, 'wakatid': 50, '。': 51, '「\\': 52, 'たい': 53, 'ため': 54, 'だ': 55, 'できる': 56, 'ない': 57, 'よって': 58, 'を': 59, '使用': 60, '文字': 61, '次に': 62, '省く': 63, '邪魔': 64, '関数': 65}


In [15]:
texts_tfidf = [] # id -> 単語表示に変えた文書ごとのTF-IDF
for doc in corpus_list_tfidf:
    text_tfidf = []
    for word in doc:
        text_tfidf.append([corpus_dic[word[0]],word[1]])
    texts_tfidf.append(text_tfidf)

In [16]:
for text in texts_tfidf:
    print(text)

[['(', 0.29477647703525461], [')', 0.29477647703525461], ['.', 0.29477647703525461], ['1', 0.29477647703525461], ['NEologd', 0.29477647703525461], ['Parse', 0.14738823851762731], ['_', 0.14738823851762731], ['tagger', 0.29477647703525461], ['text', 0.14738823851762731], ['が', 0.061171645937469793], ['て', 0.14738823851762731], ['で', 0.12234329187493959], ['など', 0.14738823851762731], ['れる', 0.14738823851762731], ['単語', 0.061171645937469793], ['原形', 0.14738823851762731], ['各', 0.29477647703525461], ['品詞', 0.14738823851762731], ['行', 0.29477647703525461], ['表示', 0.14738823851762731], ['連続', 0.29477647703525461]]
[['Parse', 0.14113121647834134], ['て', 0.14113121647834134], ['で', 0.11714949431470387], ['単語', 0.058574747157351933], ['()', 0.28226243295668269], ['1つ', 0.28226243295668269], ['\\', 0.14113121647834134], ['n', 0.14113121647834134], ['「', 0.14113121647834134], ['」', 0.058574747157351933], ['いる', 0.28226243295668269], ['た', 0.14113121647834134], ['なる', 0.28226243295668269], ['に', 0

In [22]:
for doc in corpus_list_tfidf:
    text_tfidf = []
    for word in doc:
        text_tfidf.append(corpus_dic[word[0]])
                           
text_tfidf

['_',
 'text',
 'が',
 'で',
 '」',
 'は',
 'リスト',
 '型',
 '文字列',
 't',
 "')",
 "('\\",
 'split',
 'wakatid',
 '。',
 '「\\',
 'たい',
 'ため',
 'だ',
 'できる',
 'ない',
 'よって',
 'を',
 '使用',
 '文字',
 '次に',
 '省く',
 '邪魔',
 '関数']

In [ ]:
#tfidf_df = pd.DataFrame(wakati_tfidf_matrix.toarray().T, index=text_wakati_df['内容'],  columns=wakati_words_list)
#tfidf_df